In [51]:
#import modules
import lxml.etree as etree
import gensim
import spacy
import glob
import copy
import pickle
from gensim.test.utils import get_tmpfile, common_texts
import re

### If data is not preprocessed yet

In [2]:
hbbw_xmls = glob.glob('/home/phillip/Documents/university/phd/bullinger/bullinger-korpus/data/editions/hbbw/*/*.xml')
vbs_xmls = glob.glob('/home/phillip/Documents/university/phd/bullinger/bullinger-korpus/data/editions/vbs/*/*.xml')
transcribed_xmls = glob.glob('/home/phillip/Documents/university/phd/bullinger/bullinger-korpus/data/non_edited/transcribed/*.xml')

In [3]:
all_xmls = hbbw_xmls + vbs_xmls + transcribed_xmls
len(all_xmls)

8640

In [4]:
!pip3 install https://huggingface.co/latincy/la_core_web_lg/resolve/main/la_core_web_lg-any-py3-none-any.whl

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.1/237.1 MB 13.0 MB/s eta 0:00:00m eta 0:00:010:00:01
  Obtaining dependency information for spacy<3.7.0,>=3.6.0 from https://files.pythonhosted.org/packages/58/93/051fc55f3e4e6d0d97beb77bbd5cd9e17589a0092305b4ae6164c6dcb58a/spacy-3.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached spacy-3.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (25 kB)
Using cached spacy-3.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.6 MB)
DEPRECATION: celery 4.4.7 has a non-standard dependency specifier pytz>dev. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of celery or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pyth

In [7]:
nlp = spacy.load('la_core_web_trf')

In [9]:
texts = []
for i, xml in enumerate(all_xmls):
    if i % 300 == 0:
        print('I am at file {}'.format(i))
    text = []
    la_sents = 0
    xml_open = etree.parse(open(xml, 'r'))
    xml_name = xml.split('/')[-1].replace('.xml', '') 
    root = xml_open.getroot()
    sentences = root.findall('.//s')
    for s in sentences:
        if s.get('lang') == 'la':
            s_copy = copy.deepcopy(s)
            for fl in s_copy.findall('.//fl'):
                fl.clear()
                try:
                    s_copy.remove(fl)
                except:
                    s_copy == s_copy
            
            la_sents += 1
            try:
                doc = nlp("".join(s_copy.itertext()).replace('[', '').replace(']', ''))
                lemmas = [w.lemma_ for w in doc if w.pos_ in ['NOUN']]
                text += lemmas
            except TypeError:
                continue
    if la_sents >= 5:
        texts.append([xml_name, 'latin', text])

I am at file 0
I am at file 300
I am at file 600
I am at file 900
I am at file 1200
I am at file 1500
I am at file 1800
I am at file 2100
I am at file 2400
I am at file 2700
I am at file 3000
I am at file 3300
I am at file 3600
I am at file 3900
I am at file 4200
I am at file 4500
I am at file 4800
I am at file 5100
I am at file 5400
I am at file 5700
I am at file 6000
I am at file 6300
I am at file 6600
I am at file 6900
I am at file 7200
I am at file 7500
I am at file 7800
I am at file 8100
I am at file 8400


In [10]:
texts[:2]

[['05_606_10644',
  'latin',
  ['Doctor',
   'cornmendatio',
   'uir',
   'res',
   'laus',
   'autoritati',
   'res',
   'ludus',
   'nonae',
   'iulius',
   'Bullingerus',
   'contionator',
   'praeceptor',
   'dominus']],
 ['05_642_10680',
  'latin',
  ['Salus',
   'Iuditium',
   'baron',
   'studium',
   'dux',
   'Frater',
   'ditio',
   'res',
   'concordia',
   'Capitone',
   'epistola',
   'Augustanus',
   'epistola',
   'fausta',
   'concordia',
   'epistolas',
   'dominum',
   'ecclesia',
   'pestis',
   'dominus',
   'Pellicanus',
   'LeonemTheodorum',
   'res',
   'te',
   'cum',
   'opera',
   'functio',
   'mens',
   'dominus',
   'operarius',
   'messis',
   'Carinus',
   'causa',
   'domus',
   'september',
   'Z.',
   'Dominus',
   'Bullingerus',
   'ecclesia',
   'frater']]]

In [11]:
len(texts)

7374

In [12]:
# texts all in a list, in the form of [[{file_name: ['tok1', 'tok2', ...]}], [{file_name: ['tok1', 'tok2', ...]}], ...]
with open('bullinger-letters-tok-lem.list', 'wb') as file:
    pickle.dump(texts, file)

### If data is already pre-processed and in the format mentioned above

In [21]:
texts = pickle.load(open('bullinger-letters-tok-lem.list', 'rb'))

In [23]:
values = [t[2] for t in texts]

In [26]:
texts[2]

['05_512_10548',
 'latin',
 ['concordia',
  'negotium',
  'Bernat',
  'deum',
  'morbus',
  'ecclesia',
  'morbus',
  'ἅπαντες',
  'animus',
  'conspiratio',
  'res',
  'ueritas',
  'dominus',
  'conatus',
  'pax',
  'ecclesia',
  'meritum',
  'ianuarius',
  'Grynaeus',
  'dominus',
  'Bullingerus',
  'amicus']]

In [25]:
dictionary = gensim.corpora.Dictionary(values)
corpus = [dictionary.doc2bow(doc) for doc in values]

In [29]:
# also save corpus as a mallet format
output_fname = get_tmpfile("bullinger-letters-tok-lem.mallet")
gensim.corpora.MalletCorpus.serialize(output_fname, corpus, dictionary)

In [34]:
# Train LDA model
num_topics = 100  # You can adjust this based on your data and requirements
passes = 200
lda = gensim.models.LdaMulticore(corpus, id2word=dictionary, num_topics=num_topics, passes=passes, workers=16)

# Print the topics
for idx, topic in lda.print_topics(-1):
    print(f"Topic: {idx} \nWords: {topic}\n")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [35]:
lda.save("models/gensim_lda_model-bullinger_nouns_50topics/gensim_lda_model-bullinger_nouns_{}topics_{}passes".format(num_topics, passes))

In [37]:
# Train a HDP model
hdp = gensim.models.HdpModel(corpus, id2word=dictionary)


In [39]:
hdp.print_topics(num_topics=20, num_words=10)

[(0,
  '0.019*dominus + 0.019*ecclesia + 0.018*res + 0.018*uir + 0.015*litera + 0.015*frater + 0.010*nihil + 0.009*animus + 0.008*homo + 0.008*Bullingerus'),
 (1,
  '0.021*dominus + 0.020*ecclesia + 0.018*res + 0.018*uir + 0.015*litera + 0.015*frater + 0.010*nihil + 0.009*Bullingerus + 0.009*animus + 0.008*gratia'),
 (2,
  '0.016*dominus + 0.015*uir + 0.014*frater + 0.013*res + 0.012*ecclesia + 0.011*litera + 0.009*Bullingerus + 0.007*nihil + 0.007*animus + 0.007*domino'),
 (3,
  '0.013*uir + 0.013*dominus + 0.011*litera + 0.011*frater + 0.011*res + 0.010*ecclesia + 0.008*Bullingerus + 0.007*animus + 0.006*nihil + 0.006*dies'),
 (4,
  '0.011*frater + 0.010*uir + 0.010*dominus + 0.009*res + 0.009*ecclesia + 0.008*litera + 0.007*Bullingerus + 0.006*animus + 0.005*nihil + 0.005*domino'),
 (5,
  '0.005*dominus + 0.004*uir + 0.003*frater + 0.003*res + 0.003*Bullingerus + 0.003*ecclesia + 0.003*annus + 0.003*animus + 0.002*litera + 0.002*homo'),
 (6,
  '0.005*frater + 0.004*dominus + 0.003*d

In [40]:
# try NMF

nmf = gensim.models.Nmf(corpus, num_topics=50)

In [69]:
for topic in nmf.show_topics(num_topics=50, num_words=10):
    topic_copy = topic[1]
    ids = re.findall(r'"\d{1,5}"', topic_copy)
    for wid in ids:
        wid = int(wid.replace('"', ''))
        word = dictionary[wid]
        topic_copy = topic_copy.replace('"' + str(wid) + '"', '"' + word + '"')
    print(topic[0], ' ', topic_copy)

0   0.077*"lex" + 0.030*"uerbum" + 0.021*"matrimonium" + 0.018*"uxor" + 0.015*"locus" + 0.014*"rex" + 0.014*"ius" + 0.014*"gradus" + 0.011*"coniugium" + 0.011*"liber"
1   0.160*"deus" + 0.042*"homo" + 0.034*"filius" + 0.031*"gratia" + 0.024*"dominus" + 0.016*"pater" + 0.016*"opus" + 0.015*"peccatum" + 0.015*"natura" + 0.014*"causa"
2   0.063*"minister" + 0.032*"ecclesia" + 0.028*"domino" + 0.023*"concordia" + 0.023*"nomen" + 0.022*"ecclesię" + 0.022*"nihil" + 0.021*"animus" + 0.020*"modus" + 0.019*"doctrina"
3   0.084*"nihil" + 0.026*"senatus" + 0.025*"gratia" + 0.021*"dominus" + 0.021*"consul" + 0.018*"dies" + 0.017*"Bullingerus" + 0.014*"miles" + 0.014*"exercitus" + 0.013*"amicus"
4   0.136*"fides" + 0.054*"homo" + 0.026*"locus" + 0.023*"res" + 0.020*"uita" + 0.020*"sententia" + 0.017*"nomen" + 0.017*"ratio" + 0.017*"sanguis" + 0.016*"gratia"
5   0.052*"legatus" + 0.040*"res" + 0.028*"pars" + 0.027*"colloquium" + 0.018*"homo" + 0.015*"sententia" + 0.013*"dies" + 0.011*"causa" + 0.011

In [50]:
dictionary[197]

'legatus'

In [57]:
tstring = '0.064*"621" + 0.041*"197" + 0.024*"227" + 0.024*"117" + 0.018*"11" + 0.014*"135" + 0.013*"191" + 0.012*"365" + 0.012*"3513" + 0.012*"320"'
ids = re.findall(r'"\d{1,5}"', tstring)
ids

['"621"',
 '"197"',
 '"227"',
 '"117"',
 '"11"',
 '"135"',
 '"191"',
 '"365"',
 '"3513"',
 '"320"']